In [1]:
# !wget https://malaya-dataset.s3-ap-southeast-1.amazonaws.com/crawler/academia/academia-pdf.json

In [2]:
import json
import cleaning
from tqdm import tqdm

In [3]:
with open('academia-pdf.json') as fopen:
    pdf = json.load(fopen)
    
len(pdf)

224

In [4]:
from unidecode import unidecode

def clean(string):
    string = [cleaning.cleaning(s) for s in string]
    
    string = [s.strip() for s in string if 'tarikh' not in s.lower() and 'soalan no' not in s.lower()]
    string = [s for s in string if not ''.join(s.split()[:1]).isdigit() and '.soalan' not in s.lower() and 'jum ' not in s.lower()]
    string = [s for s in string if not s[:3].isdigit() and not s[-3:].isdigit()]
    return string

In [5]:
outer = []

for k in tqdm(range(len(pdf))):

    c = clean(pdf[k]['content']['content'].split('\n'))
    t, last = [], 0

    i = 0
    while i < len(c):
        text = c[i]

        if len(text) > 5:
            if len(text.split()) > 1:
                t.append(text)
            last = i
        else:
            if len(t) and (i - last) > 2:
                t.append('')
                outer.extend(t)
                t = []
                last = i
            elif not len(t):
                last = i

        i += 1
    
    if len(t):
        t.append('')
        outer.extend(t)

100%|██████████| 224/224 [00:31<00:00,  7.15it/s]


In [6]:
len(outer)

651957

In [7]:
%%time

temp_vocab = list(set(cleaning.multiprocessing(outer, cleaning.unique_words)))

CPU times: user 462 ms, sys: 216 ms, total: 678 ms
Wall time: 772 ms


In [8]:
%%time

# important
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.duplicate_dots_marks_exclamations, list_mode = False)
print(len(temp_dict))

881
CPU times: user 104 ms, sys: 111 ms, total: 215 ms
Wall time: 655 ms


In [9]:
outer = cleaning.string_dict_cleaning(outer, temp_dict)

100%|██████████| 651957/651957 [00:02<00:00, 282625.60it/s]


In [10]:
%%time

# important
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.remove_underscore, list_mode = False)
print(len(temp_dict))

71
CPU times: user 113 ms, sys: 117 ms, total: 230 ms
Wall time: 356 ms


In [11]:
outer = cleaning.string_dict_cleaning(outer, temp_dict)

100%|██████████| 651957/651957 [00:02<00:00, 281336.39it/s]


In [12]:
%%time

# important
temp_dict = cleaning.multiprocessing(outer, cleaning.isolate_spamchars, list_mode = False)
print(len(temp_dict))

0
CPU times: user 223 ms, sys: 129 ms, total: 351 ms
Wall time: 928 ms


In [13]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.break_short_words, list_mode = False)
print(len(temp_dict))

2631
CPU times: user 112 ms, sys: 108 ms, total: 220 ms
Wall time: 258 ms


In [14]:
outer = cleaning.string_dict_cleaning(outer, temp_dict)

100%|██████████| 651957/651957 [00:02<00:00, 273863.82it/s]


In [15]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.break_long_words, list_mode = False)
print(len(temp_dict))

765
CPU times: user 90.7 ms, sys: 116 ms, total: 207 ms
Wall time: 261 ms


In [16]:
outer = cleaning.string_dict_cleaning(outer, temp_dict)

100%|██████████| 651957/651957 [00:02<00:00, 284637.14it/s]


In [17]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.remove_ending_underscore, list_mode = False)
print(len(temp_dict))

10
CPU times: user 104 ms, sys: 104 ms, total: 208 ms
Wall time: 323 ms


In [18]:
outer = cleaning.string_dict_cleaning(outer, temp_dict)

100%|██████████| 651957/651957 [00:02<00:00, 271179.24it/s]


In [19]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.remove_starting_underscore, list_mode = False)
print(len(temp_dict))

56
CPU times: user 98.9 ms, sys: 107 ms, total: 206 ms
Wall time: 323 ms


In [20]:
outer = cleaning.string_dict_cleaning(outer, temp_dict)

100%|██████████| 651957/651957 [00:02<00:00, 279244.09it/s]


In [21]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.end_punct, list_mode = False)
print(len(temp_dict))

138699
CPU times: user 510 ms, sys: 124 ms, total: 634 ms
Wall time: 723 ms


In [22]:
outer = cleaning.string_dict_cleaning(outer, temp_dict)

100%|██████████| 651957/651957 [00:02<00:00, 251827.17it/s]


In [23]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.start_punct, list_mode = False)
print(len(temp_dict))

37464
CPU times: user 211 ms, sys: 152 ms, total: 363 ms
Wall time: 380 ms


In [24]:
outer = cleaning.string_dict_cleaning(outer, temp_dict)

100%|██████████| 651957/651957 [00:02<00:00, 242230.75it/s]


In [25]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.join_dashes, list_mode = False)
print(len(temp_dict))

65
CPU times: user 89.4 ms, sys: 144 ms, total: 233 ms
Wall time: 285 ms


In [26]:
outer = cleaning.string_dict_cleaning(outer, temp_dict)

100%|██████████| 651957/651957 [00:02<00:00, 256821.01it/s]


In [27]:
outer[-100:]

['Indonesia : Dar Ihya al kutub , h . 140-141 jw 1,.*Tim Penyusun ( 2003 ), Ensiklopedi Islam untuk pelajar . Kuala',
 'Lumpur : Era Visi Publication , 2003 h.75 .*\'" Tahir Mahmood ( 1993 ), Human Rights in Islamic Law.New',
 "Delhi : Institute of Objektive Studies , h , 4-11.Abul ' ala al -",
 'Mahududi ( 1985 ), Hak asasi manusia dalam Islam . Bandung :',
 "Pustaka h . l9-20 .*- Abdul Wahab Khalaf ( 2003 ), Siasah Syari'ah dalam",
 'pemerintahan Islan . Kuala Lumpur : al-Hidayah',
 "*'Republika , 07 Jmuai 2009 ** Tim Penlusun . ( 1999 ) Ensiklopedi Hukum Islam . Jakarta :",
 '\' o"iii Al-Imam Muhammad Ab,t Zahrah ( 1424 . H ) usut Fikh .',
 '* r Ibn Katsir ( 2004 ) Tafsir lbn Katsir . Fiiyadh : Dr Salm , jilid 2 ,',
 'h . 835n Hu*kr , (1990 ) Tafsir al-Azhar , jilid 6 . Singapura : Pustaka',
 'n Ibn Hajr a1-Asqhalani ( 2004 ) Bulughul Marom . fuyadh : Dar us -',
 "'riiiM . ali Ash-Shabuni ( 1994 ) Tafsir Ayat-Ayat Hukum Dalam al -",
 "Qur'an (terj ). Bandung : aLma'ari { h . 290il '

In [28]:
with open('dumping-pdf.txt', 'w') as fopen:
    fopen.write('\n'.join(outer))